In [1]:
import os
import pandas as pd
from pydantic import BaseModel, create_model
from openai import OpenAI

# 设置代理环境（如果需要）
os.environ["http_proxy"] = "127.0.0.1:7890"
os.environ["https_proxy"] = "127.0.0.1:7890"

# config.py
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()

# 读取
ZetaTechs_api_key = os.getenv('ZetaTechs_api_key')
ZetaTechs_api_base = os.getenv('ZetaTechs_api_base')

client = OpenAI(api_key=ZetaTechs_api_key, base_url=ZetaTechs_api_base)

# 读取 CSV 文件并忽略空列，保留“采集时间”和“备注【疑问汇总】”
def load_csv(file_path):
    df = pd.read_csv(file_path)
    # 找到不为空的列名
    non_empty_columns = df.columns[df.notna().any()].tolist()
    # 加入需要保留的列
    required_columns = ["采集来源", "来源链接", "采集时间", "备注【疑问汇总】"]
    final_columns = [col for col in non_empty_columns if col not in required_columns] + required_columns
    # 根据有效列筛选数据，并保留“采集时间”和“备注【疑问汇总】”列
    df = df[final_columns]
    return df

# 自动生成 column_mapping，忽略 "采集时间" 和 "备注【疑问汇总】"
def generate_column_mapping(df):
    columns_to_include = df.columns[:-2]  # 忽略最后两列
    column_mapping = {col: col for col in columns_to_include}
    return column_mapping

# 创建大模型的输入
def create_model_input(df, column_mapping):
    input_data = []
    for _, row in df.iterrows():
        mapped_input = {column_mapping[key]: row[key] for key in column_mapping}
        input_data.append(mapped_input)
    return input_data

def create_system_prompt():
    return """
    You are an expert in data augmentation. You will be provided with a table structure and sample data. 
    Your task is to augment the dataset by generating new entries while maintaining consistency with the original format and meaning of the columns.
    Ensure that each generated entry follows the exact column order and data type of the original dataset.
    Focus on creating diverse and realistic entries, ensuring the augmented data aligns closely with the structure and content of the original dataset.
    """

In [4]:
# 不包含对列名的说明，仅仅是现有数据
def create_user_prompt_1(input_data):
    # 构建user prompt，将所有input_data中的信息加入提示
    prompt = "Here is the structure of the dataset with sample data:\n\n"
    for row_idx, row in enumerate(input_data):
        prompt += f"Row {row_idx + 1}:\n"
        for col_name, value in row.items():
            prompt += f"  Column: {col_name}, Value: {value}\n"
        prompt += "\n"  # 每行数据后增加换行，区分不同行
    prompt += "Please generate more data in the same structure and format."
    return prompt

# 对每个列名进行说明的prompt
def create_user_prompt_2(input_data):
    prompt = "Here is the structure of the dataset with sample data and column descriptions:\n\n"
    columns = list(input_data[0].keys())
    for col in columns:
        prompt += f"Column: {col}\n"
        prompt += f"Description: [Add a brief description of what this column represents]\n"
        prompt += f"Example: {input_data[0][col]}\n\n"
    prompt += "Please generate more data in the same structure and format, ensuring each column contains appropriate data."
    return prompt

# 综合上述两个prompt
def create_user_prompt_3(input_data):
    prompt = "Here is the structure of the dataset with column descriptions and sample data:\n\n"
    
    # 列描述
    columns = list(input_data[0].keys())
    for col in columns:
        prompt += f"Column: {col}\n"
        if col == "年月":
            prompt += "Description: The date in format YYYY/MM/DD\n"
        elif col == "地区":
            prompt += "Description: The name of the region or province\n"
        elif col == "健身房人数":
            prompt += "Description: Number of people using gyms\n"
        elif col == "单位":
            prompt += "Description: Unit of measurement (e.g., 万人, 个, etc.)\n"
        elif col == "健身工作室数量":
            prompt += "Description: Number of fitness studios\n"
        elif col == "健身俱乐部数量":
            prompt += "Description: Number of fitness clubs\n"
        elif col == "健身渗透率":
            prompt += "Description: Fitness penetration rate (as a decimal)\n"
        elif col == "经常参加体育锻炼的人数":
            prompt += "Description: Number of people regularly participating in sports\n"
        elif col == "人均体育场馆面积":
            prompt += "Description: Average sports facility area per person\n"
        elif col == "健身休闲产业规模":
            prompt += "Description: Scale of the fitness and leisure industry\n"
        elif col == "马拉松参加人数":
            prompt += "Description: Number of marathon participants\n"
        elif col == "采集来源":
            prompt += "Description: Source of data collection\n"
        elif col == "来源链接":
            prompt += "Description: URL link to the data source\n"
        else:
            prompt += "Description: [Add a brief description of what this column represents]\n"
        prompt += f"Example: {input_data[0][col]}\n\n"

    # 现有数据
    prompt += "Sample data:\n\n"
    for row_idx, row in enumerate(input_data):
        prompt += f"Row {row_idx + 1}:\n"
        for col_name, value in row.items():
            prompt += f"  Column: {col_name}, Value: {value}\n"
        prompt += "\n"  # 每行数据后增加换行，区分不同行

    prompt += "Please generate more data in the same structure and format, ensuring each column contains appropriate and realistic data. Maintain consistency with the provided examples and descriptions."
    return prompt

In [5]:
# test load_csv() and generate_column_mapping() and create_model_input()
file_path = "8-现代时尚 - 健身人数信息.csv"

# 第1步：加载 CSV 文件
df = load_csv(file_path)
df_columns = df.columns[:-2]

# 第2步：自动生成 column_mapping，忽略 "采集时间" 和 "备注【疑问汇总】"
column_mapping = generate_column_mapping(df)

# 第3步：创建大模型输入
input_data = create_model_input(df, column_mapping)

# 第4步：创建系统提示和用户提示
system_prompt = create_system_prompt()
user_prompt_1 = create_user_prompt_1(input_data)
print(user_prompt_1)

Here is the structure of the dataset with sample data:

Row 1:
  Column: 年月, Value: 2023-12-01
  Column: 地区, Value: 中国
  Column: 健身房人数, Value: 6975.0
  Column: 单位, Value: 万人
  Column: 健身工作室数量, Value: 42177.0
  Column: 单位.1, Value: 个
  Column: 健身俱乐部数量, Value: 36447.0
  Column: 单位.2, Value: 个
  Column: 健身渗透率, Value: 0.0502
  Column: 经常参加体育锻炼的人数, Value: 21000.0
  Column: 单位.3, Value: 万人
  Column: 人均体育场馆面积, Value: 1.5
  Column: 单位.4, Value: 平方米
  Column: 健身休闲产业规模, Value: 675.0
  Column: 单位.5, Value: 亿元
  Column: 马拉松参加人数, Value: 50
  Column: 单位.6, Value: 万人
  Column: 采集来源, Value: 中商产业研究院
  Column: 来源链接, Value: https://www.askci.com/news/chanye/20240311/175756271015107566247316.shtml

Row 2:
  Column: 年月, Value: 2022-12-01
  Column: 地区, Value: 中国
  Column: 健身房人数, Value: 7145.0
  Column: 单位, Value: 万人
  Column: 健身工作室数量, Value: 45529.0
  Column: 单位.1, Value: 个
  Column: 健身俱乐部数量, Value: 39620.0
  Column: 单位.2, Value: 个
  Column: 健身渗透率, Value: 0.051
  Column: 经常参加体育锻炼的人数, Value: 21500.0
  Column

In [6]:
user_prompt_2 = create_user_prompt_2(input_data)
print(user_prompt_2)

Here is the structure of the dataset with sample data and column descriptions:

Column: 年月
Description: [Add a brief description of what this column represents]
Example: 2023-12-01

Column: 地区
Description: [Add a brief description of what this column represents]
Example: 中国

Column: 健身房人数
Description: [Add a brief description of what this column represents]
Example: 6975.0

Column: 单位
Description: [Add a brief description of what this column represents]
Example: 万人

Column: 健身工作室数量
Description: [Add a brief description of what this column represents]
Example: 42177.0

Column: 单位.1
Description: [Add a brief description of what this column represents]
Example: 个

Column: 健身俱乐部数量
Description: [Add a brief description of what this column represents]
Example: 36447.0

Column: 单位.2
Description: [Add a brief description of what this column represents]
Example: 个

Column: 健身渗透率
Description: [Add a brief description of what this column represents]
Example: 0.0502

Column: 经常参加体育锻炼的人数
Descriptio

In [7]:
user_prompt_3 = create_user_prompt_3(input_data)
print(user_prompt_3)

Here is the structure of the dataset with column descriptions and sample data:

Column: 年月
Description: The date in format YYYY/MM/DD
Example: 2023-12-01

Column: 地区
Description: The name of the region or province
Example: 中国

Column: 健身房人数
Description: Number of people using gyms
Example: 6975.0

Column: 单位
Description: Unit of measurement (e.g., 万人, 个, etc.)
Example: 万人

Column: 健身工作室数量
Description: Number of fitness studios
Example: 42177.0

Column: 单位.1
Description: [Add a brief description of what this column represents]
Example: 个

Column: 健身俱乐部数量
Description: Number of fitness clubs
Example: 36447.0

Column: 单位.2
Description: [Add a brief description of what this column represents]
Example: 个

Column: 健身渗透率
Description: Fitness penetration rate (as a decimal)
Example: 0.0502

Column: 经常参加体育锻炼的人数
Description: Number of people regularly participating in sports
Example: 21000.0

Column: 单位.3
Description: [Add a brief description of what this column represents]
Example: 万人

Column: 

### user_prompt_4 - 目前感觉最有前景的prompt

### **TableStructureManager的主要有三个功能:**
#### 1. get_column_description(self, main_category, sub_category, column_name)
#### 2. add_or_update_columns(self, main_category, sub_category, columns_dict)
#### 3. get_all_columns(self, main_category, sub_category)

### **test TableStructureManager**

In [26]:
from table_structure_manager import TableStructureManager

# 创建 TableStructureManager 实例
manager = TableStructureManager()

# 调用 load_structures 方法
structures = manager.load_structures()
print("Loaded structures:", type(structures), structures.keys(), "\n\n", structures.values())

Loaded structures: <class 'dict'> dict_keys(['8-现代时尚']) 

 dict_values([{'健身人数信息': {'年月': 'The date in format YYYY/MM/DD', '地区': 'The name of the region or province', '健身房人数': 'Number of people using gyms', '单位': 'Unit of measurement (e.g., 万人, 个, etc.)', '健身工作室数量': 'Number of fitness studios', '健身俱乐部数量': 'Number of fitness clubs', '健身渗透率': 'Fitness penetration rate (as a decimal)', '经常参加体育锻炼的人数': 'Number of people regularly participating in sports', '人均体育场馆面积': 'Average sports facility area per person', '健身休闲产业规模': 'Scale of the fitness and leisure industry', '马拉松参加人数': 'Number of marathon participants', '采集来源': 'Source of data collection', '来源链接': 'URL link to the data source', '采集时间': 'Time of data collection', '备注【疑问汇总】': 'Remarks and questions summary'}, '服装企业渠道分析': {'年月': 'The date in format YYYY/MM/DD', '企业': 'Name of the company', '直营收入占比': 'Proportion of direct sales revenue', '经销收入占比': 'Proportion of distribution revenue', '电商收入占比': 'Proportion of e-commerce revenue', '前五大客户占

In [27]:
# 调用 get_column_description 方法
description = manager.get_column_description("8-现代时尚", "健身人数信息", "年月")
print("Column description:", description)

Column description: The date in format YYYY/MM/DD


In [28]:
# 调用 add_or_update_column 方法
# 添加或更新多个列
new_columns = {
    "新列名1": "新列1的描述",
    "新列名2": "新列2的描述",
    "新列名3": "新列3的描述"
}

manager.add_or_update_columns("8-现代时尚", "新表名", new_columns)
print("Updated structures:", manager.structures)

Updated structures: {'8-现代时尚': {'健身人数信息': {'年月': 'The date in format YYYY/MM/DD', '地区': 'The name of the region or province', '健身房人数': 'Number of people using gyms', '单位': 'Unit of measurement (e.g., 万人, 个, etc.)', '健身工作室数量': 'Number of fitness studios', '健身俱乐部数量': 'Number of fitness clubs', '健身渗透率': 'Fitness penetration rate (as a decimal)', '经常参加体育锻炼的人数': 'Number of people regularly participating in sports', '人均体育场馆面积': 'Average sports facility area per person', '健身休闲产业规模': 'Scale of the fitness and leisure industry', '马拉松参加人数': 'Number of marathon participants', '采集来源': 'Source of data collection', '来源链接': 'URL link to the data source', '采集时间': 'Time of data collection', '备注【疑问汇总】': 'Remarks and questions summary'}, '服装企业渠道分析': {'年月': 'The date in format YYYY/MM/DD', '企业': 'Name of the company', '直营收入占比': 'Proportion of direct sales revenue', '经销收入占比': 'Proportion of distribution revenue', '电商收入占比': 'Proportion of e-commerce revenue', '前五大客户占比': 'Proportion of top five customers', '

In [29]:
# 调用 get_all_columns 方法
columns = manager.get_all_columns("8-现代时尚", "新表名")
print("All columns in Category1 -> SubCategory1:", columns)

All columns in Category1 -> SubCategory1: {'新列名1': '新列1的描述', '新列名2': '新列2的描述', '新列名3': '新列3的描述'}
